In [ ]:
# install colab dependencies, if you are running this script on your machine, please build a new conda enviroment
# and install the dependices marked at requirements.txt
!conda install gxx_linux-64 gcc_linux-64 swig
!pip install -r requirements.txt
import smac
import torch

# Handout session1- Bayesian Optimization


In this handout session, we will focus on a simple yet important application of BO for deep learning: tunning the learning rate of a deep neural network. It is highly recommended to learn a bit about Bayesian Optimization from the [MOOC](https://ki-campus.org/node/109) to better understand the concepts and methods discussed in the following sections.

First, let's see how learning rates influence the performance of a neural network. We first create a simple network that will be trained on [KMNIST](https://github.com/rois-codh/kmnist) dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
from typing import List, Any
import typing
import os
import numpy as np

from func_eval import LeNet_Evaluator


torch.manual_seed(1)

dataset_path = os.path.join('tmp', 'dataset')
# you need to specify your device (cpu or cuda) here.
device = torch.device('cpu')


The Class "LeNet_Evaluator" is a helper class that allows you to evaluate a configuration without knowing the details of training a neural network. The most import functions that will be used in this exercise are `__init__` and `eval_config`

In [ ]:
help(LeNet_Evaluator.__init__)

In [ ]:
help(LeNet_Evaluator.eval_config)

Now we are creating an evaluator object

In [ ]:
evaluator = LeNet_Evaluator(dataset_path, device)

An AutoML system needs to know what it optimizes for. A configuration space contains all the possible hyperparameters that an AutoML system can evaluate. Here we use the package [ConfigSpace](https://automl.github.io/ConfigSpace/master/index.html) to manage our configuration space. ConfigSpace allows us to define different sorts of hyperparameters that need to be optimized in AutoML problems. The values that these hyperparameters take can be of different types. Here, we restrict the space by considering Float and Integer values sampled from a unifrom distribution, which is imported through [UniformFloatHyperparameter](https://automl.github.io/ConfigSpace/master/API-Doc.html#ConfigSpace.hyperparameters.UniformFloatHyperparameter) and [UniformIntegerHyperparameter](https://automl.github.io/ConfigSpace/master/API-Doc.html#ConfigSpace.hyperparameters.UniformIntegerHyperparameter).

In [ ]:
from ConfigSpace import ConfigurationSpace, Configuration
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, UniformIntegerHyperparameter, Constant

# we build a configuration space from which a configuration could be sampled 
cs_lenet = ConfigurationSpace(seed=1)

batch_size = UniformIntegerHyperparameter('batch_size', lower=32, upper=1024, default_value=512, log=True)
learning_rate = UniformFloatHyperparameter('learning_rate', lower=1e-4, upper=1., default_value=1e-2, log=True)

cs_lenet.add_hyperparameters([batch_size, learning_rate])

Let's first evaluate the hyperparameteres with a manually selected hyperparameter configuration. You could select the hyperparameter configuration that you believe optimal for this task

In [ ]:
#TODO select the corresponding hyperparameter values and store them as a dictionary
hps_values_hand_tuned = {'batch_size': 128,
                        'learning_rate': 1e-4}



Now, we use the hand-tuned values and store them as a Configuration space object. We then evaluate this configuration.

In [ ]:
# generate a Configuration object 
cfg_hand_tunned = Configuration(configuration_space=cs_lenet, values=hps_values_hand_tuned)
val_score_hand_tunned = evaluator.eval_config(cfg_hand_tunned)

## Using SMAC to do hyperparameter optimization


[SMAC](https://github.com/automl/SMAC3) is a tool for algorithm configuration to optimize the parameters of arbitrary algorithms, including hyperparameter optimization of Machine Learning algorithms. SMAC also supports more advanced hyperparameter optimization technique such as multi-fidelity optimization process. Please follow the [examples](https://github.com/automl/SMAC3/tree/master/examples/python) and optimize the batch size and learning rate with SMAC for 50 evaluations and compare the reuslt with a hand-tuned configuration (if appliable). You could use the facade [SMAC4HPO](https://github.com/automl/SMAC3/blob/master/examples/python/svm_cv.py) or [SMAC4MF](https://github.com/automl/SMAC3/blob/master/examples/python/mlp_mf.py) 

Doing a full HPO loop might take too much time (around 1.5 hours). If you successfully implemented this section, you could simply let this script run on the backend and continue the following exercises.

Hint: `SMAC4HPO` usually requires one input : `cfg`. You could use `functools.partial` to specify other inputs. However, `SMAC4MF` requires `budget` as well.   

In [ ]:
# TODO optimize the batch size and learning rate with SMAC.

# (Optional) Using DEHB to do hyperparameter optimization

Alternative to SMAC, [DEHB](https://github.com/automl/DEHB) is yet another scalable, robust and efficient hyperparameter optimization. Please follow the [examples](https://github.com/automl/DEHB/blob/master/examples/02_using%20DEHB_without_ConfigSpace.ipynb) and optimize the batch size and learning rate with DEHB for 3600 seconds and compare the reuslt with SMAC (if appliable). As before, you could simply let this script run on the backend and continue the following exercises.

In [ ]:
# We create a wrapper for DEHB here
import time
def dehb_wrapper_eval_configuration(cfg, budget):
    start_time = time.time()
    res = evaluator.eval_config(cfg=cfg, budget=budget)
    cost = time.time() - start_time
    result = {
        "fitness": res,  # DE/DEHB minimizes
        "cost": cost,
    }
    return result

In [ ]:
# Since DEHB does not provide a setup.py file, you need to manually export the path of DEHB to PYTHONPATH:
import sys
import os
from functools import partial
dehb_path = './DEHB'
os.environ['PATH'] += ':'+dehb_path
from functools import partial

from dehb import DEHB

# TODO use DEHB to optimize the 2 hyperparameters

# Bayesian Optimization

As we would expect, we could probably do better by further manually tunning these hyperparameters. Let's automate the tunning process with Bayesian Optimization (BO).

BO is an iterative algorithm that works as follows: 
- It first fits a probability model ([surrogate model](#Surrogate-Model)) that represents the current belief of what an unknown function looks like based on the available data samples.
- It then uses this model to guide the optimization process by measuring the potential quality of one configuration over another, done through a heuristic called an [acquisition function](#Acquisition-Function). 
- Finally, an optimizer is used to find the configuration that will be evaluated in this round. 

## Surrogate Model

Evaluating a configuration can be an expensive job. Furthermore, we do not have any information about how the actual loss will react to different hyperparameter configurations. In the context of BO, a so called surrogate model is fitted to all the previous evaluations, while an [acquisition function](#Acquisition-Function) is employed to decide which point to evaluate next according to the prediction given by the surrogate model. A surrogate model is required to give two predictions: 
- the mean of the posterior belief distribution
- variance from this mean cross the whole space

Here we provide a uniform abstract base model that can fit the given data distribution and predict the mean and variances at the target position. 

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
np.random.seed(1)

class BaseModel(object):
    """
    A surrogate model is used to evaluate the potential loss distribution, and 
    so, it should be able to predict the mean and variance values on the 
    given position.
    """
    def __init__(self):
        self.is_trained = False
        self.model = None
        self.imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=np.finfo(float).max)

    def train(self, X: np.ndarray, y: np.ndarray):
        if len(X.shape) == 1:
            X = X[np.newaxis, :]
        X = self._impute_nan(X)
        self.model.fit(X, np.squeeze(y))
        self.is_trained = True

    def predict(self, X: np.ndarray, **kwargs):
        if len(X.shape) == 1:
            X = X[np.newaxis, :]
        if not self.is_trained:
            raise ValueError("model needs to be trained first!")
        
        X = self._impute_nan(X)
        
        return self._predict(X, **kwargs)

    def _predict(self, X: np.ndarray, **kwargs):
        raise NotImplementedError

    def _impute_nan(self, input):
        return self.imp.fit_transform(input)

We use a toy example to show how surrogate models describe the possibile distribution of the target function. First, let's generate a set of data points from a simple 1-D distribution. In this case, we use $f (x) = sin (5x) \big ( 1 - tanh(x ^ 2) \big )$ and then add noise to it to get our set of samples

In [ ]:
def func(x):
    """
    This is a simple function that we need to approximate using a surrogate model
    """
    return np.sin(5. * x) * (1. - np.tanh(x ** 2)) 

def noisy(y, noise_scale=0.0):
    """
    We add random gaussian noise to points sampled from the function
    """
    return y + np.random.normal(scale=noise_scale, size=y.shape)

# Bounds of the surrogate model
lower_sur = -2
upper_sur = 2

# create the X and Y axes
x_sur = np.linspace(lower_sur, upper_sur, 400).reshape(-1, 1)
fx_sur = func(x_sur)

# scale for adding the noise
noise_scale_sur = 0.2

# sample 20 points between the bounds to create a training prior,
# evaluate the function on these points, and then add noise
x_train_sur = np.linspace(lower_sur, upper_sur, 20).reshape(-1, 1)
y_train_sur = func(x_train_sur)
y_train_sur = noisy(y_train_sur, noise_scale=noise_scale_sur)

# sample 100 more points to test
x_test_sur = np.linspace(lower_sur, upper_sur, 100).reshape(-1, 1)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150


def plot_surrogate_model_regression(ax, fig, surrogate_predict_info=None):
    ax.plot(x_sur, fx_sur, "r--", label="True (unknown)")
    ax.plot(x_sur, fx_sur + 1.96 * noise_scale_sur, 'y--',label='Noise Bound')
    ax.plot(x_sur, fx_sur - 1.96 * noise_scale_sur, 'y--')

    ax.plot(x_train_sur, y_train_sur, 'k*', label="Observations")
    
    if surrogate_predict_info is not None:
        mu_sur = surrogate_predict_info['mu']
        var_sur = surrogate_predict_info['var']
        std_sur = np.sqrt(var_sur)
        
        mu_sur = np.squeeze(mu_sur)
        std_sur = np.squeeze(std_sur)

        model_name = surrogate_predict_info['model_name']
        ax.plot(x_test_sur, mu_sur, "b", label="predicted mean")
        ax.fill_between(x=np.squeeze(x_test_sur), 
                        y1=mu_sur - 1.9600 * std_sur, y2=mu_sur + 1.9600 * std_sur, 
                        alpha=.2, fc="b",
                        label="predicted uncertainty")
        ax.set_title(f'predicted result from {model_name}')
    else:
        ax.set_title(f"An unknown function")


    ax.legend()
    ax.plot()

    
def plot_model_prediction(model_type, model_kwargs: dict={}):
    model_name = model_type.__name__
    model = model_type(**model_kwargs)
    model.train(x_train_sur, y_train_sur)
    predicted_mean_sur, predicted_var_sur = model.predict(x_test_sur)

    predict_info = {'mu': predicted_mean_sur, 'var': predicted_var_sur, 'model_name': model_name}

    fig, ax = plt.subplots()
    plot_surrogate_mdoel_regression(ax, fig, predict_info)
    plt.show()


fig, ax = plt.subplots()
plot_surrogate_model_regression(ax, fig)
plt.show()

### A dummy model

We use an exemplary dummy regression model showing you how to train a surrogate model and use it to predict the potential data distribution.

In [ ]:
from sklearn.dummy import DummyRegressor

class DummyModel(BaseModel):
    """
    This is only an example showing you how to implement a surrogate model
    """
    def __init__(self, constant_value=0.5):
        super(DummyModel, self).__init__()
        self.model = DummyRegressor(strategy='constant', constant=constant_value)

    def _predict(self, X: np.ndarray, **kwargs):
        mu, std = self.model.predict(X, return_std=True)
        return mu, std ** 2

In [ ]:
plot_model_prediction(DummyModel)    

### Gaussian Processes

[Gaussian Processes](http://www.gaussianprocess.org/gpml/chapters/RW.pdf) (GPs) are one of the most popular choices of surrogate models.

A GP predicts the distribution of a target point as :
$$
f_{\star} \big ( \mathbf{X_{\star}}, \mathbf{X}, \mathbf{y} \big) \sim \mathcal{N} \big(K_{*} K_y^{-1} \mathbf{y}, \nonumber{} K_{**} - K_{*}^T K_{y}^{-1} K_{*}\big)
$$
        
where $\mathbf{X}$ and $\mathbf{y}$ are the previous observed points and values, $\mathbf{X_{\star}}$ are the points to be evaluated. $K_{*} = k \big(\mathbf{X}, \mathbf{X^\star}\big)$ denotes the vector of covariances between $\mathbf{X_{\star}}$ and all previous observations. $K_{**} = k \big(\mathbf{X^\star} \mathbf{X^\star}\big)$ is the covariance matrix of all previously evaluated configurations.

Please follow the example given in DummyModel and implement all the related function under [Surrogate Model](#Surrogate-Model). You could use your favourite packages, e.g., [Scikit-Learn](https://scikit-learn.org/stable/), [gpytorch](https://gpytorch.ai/). 


The choice of the kernel can be seen as a prior on the form of the function to be approximated. Please try different kernels and see how the regression model changes. Please explain which kernel you would actually use in practice in the context of AutoML.

After you have finished your implementation, you could plot the distribution predicted by GP

In [ ]:
import typing
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel, Kernel

class GaussianProcess(BaseModel):
    # TODO implement all the functions related to a GP. You need to consider how to initialze the model and
    # do a correct prediction



In [ ]:
# After that we could plot the mean and variance values predicted by a GP model
plot_model_prediction(GaussianProcess)

### Random Forest

Random Forests (RF) is another popular surrogate model. RF is the ensemble of multiple regression trees. Each tree in RF gives its own prediction. The final predicted mean and variance are the empirical mean and variances of these predictions.

Please build a RF model that fits the observations and predicts the mean and variance values at the quired points. You need to implement all the related function.
   
After you have finished your implementation, please run the code to plot the predicted distribution by RF

As a first step, we use the [pyrfr](https://github.com/automl/random_forest_run) package to construct a random forest model

In [ ]:
from surrogate_model import RFR
 
plot_model_prediction(RFR, model_kwargs={'do_bootstrapping': True})

Now you need to implement an RF model by yourself with [Scikit-Learn](https://scikit-learn.org/stable/) and compare the predicted result with the one predicted by pyrfr.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import copy

class RandomForest(BaseModel):
    # TODO implement all the functions related to a GP. 
    # You need to consider which values are required to initialze the model 
    # and make a correct prediction

In [ ]:
# After that we can plot the mean and variance values predicted by a RF model

plot_model_prediction(RandomForest)

## Acquisition Function

Acquisition function $u(\cdot)$ judges the utility (or usefulness) of evaluating $f$ at 
$\mathbf{\lambda}^{(t)}\in \mathbf{\Lambda}$. To this end, acquisition functions trade off exploration and exploitation. Some popular examples of acquisition functions are: 
- Probability of improvement ([PI](#Probabilitiy-of-Improvement-\(PI\))) 
- Expected improvement ([EI](#Expected-Improvement-\(EI\))) 
- Lower confidence bound ([LCB](#Lower-Confidence-Bound-\(LCB\))) 

An acquisition function usually requires the following item:
   - A surrogate model that is trained with previous evaluated data points and able to predict the mean and variance values at the target points
   - Statistical information of the previous evaluated points (e.g., the current best evaluated values in [PI](#Probabilitiy-of-Improvement-\(PI\)) and [EI](#Expected-Improvement-\(EI\)), number of previously evaluated points in [LCB](#Lower-Confidence-Bound-\(LCB\)))
   - These items need to be updated during the course of optimization (you need to specify `self._required_update` at `__init__` function)
    
We control the exploration-exploitation trade off with the hyperparameter assigned to the acquisition functions.

In [ ]:


def convert_configurations_to_array(configs: List[Configuration]):
    return np.array([config.get_array() for config in configs], dtype=np.float64)

class AbstractAcquisitionFunction(object):
    """
    An acquisition function determines how good a configuration is based on the 
    predicted result of a surrogate model
    """
    def __init__(self, model: BaseModel):
        self.model = model
        self._required_updates = ('model', )

    def update(self, **kwargs: Any) -> None:
        for key in self._required_updates:
            if key not in kwargs:
                raise ValueError(
                    'Acquisition function %s needs to be updated with key %s, but only got '
                    'keys %s.'
                    % (self.__class__.__name__, key, list(kwargs.keys()))
                )
        for key in kwargs:
            if key in self._required_updates:
                setattr(self, key, kwargs[key])

    def __call__(self, configurations: List[Configuration]) -> np.ndarray:
        X = convert_configurations_to_array(configurations)
        if len(X.shape) == 1:
            X = X[np.newaxis, :]

        acq = self._compute(X)
        if np.any(np.isnan(acq)):
            idx = np.where(np.isnan(acq))[0]
            acq[idx] = -np.finfo(np.float).max
        return acq

    def _compute(self, X: np.ndarray) -> np.ndarray:
        raise NotImplementedError()

Again, we will generate a set of data points. An off-the-shelf GP model will do the prediction task. You only need to focus on computing the acquisition function values.

In [ ]:
from surrogate_model.gaussian_process import GaussianProcess
from surrogate_model.random_forest import RFR

# set the bounds
lower_acq = -2
upper_acq = 2

# Create the configuration space and add hyperparameters
cs = ConfigurationSpace(seed=1)
cs.add_hyperparameter(UniformFloatHyperparameter("x", lower=lower_acq, upper=upper_acq))

# sample a configuration
configs_train_acq = cs.sample_configuration(8)

# Create traces for trainign
x_train_acq = np.asarray([config['x'] for config in configs_train_acq])
fx_train_acq = func(x_train_acq)
y_train_acq = noisy(fx_train_acq, noise_scale=0.04)

# Create traces for test
x_test_acq = np.linspace(lower_acq, upper_acq, 100).reshape(-1, 1)
configs_test_acq = [Configuration(configuration_space=cs, values={"x": x.item()}) for x in x_test_acq]
fx_gt_acq = func(x_test_acq)

# we use a GP model to predict the mean and variance values, feel free to use other surrogate models here
surro_model_acq = GaussianProcess()
surro_model_acq.train(convert_configurations_to_array(configs_train_acq), y_train_acq)

mu_acq, var_acq = surro_model_acq.predict(convert_configurations_to_array(configs_test_acq))
std_acq = np.sqrt(var_acq)
mu_acq = np.squeeze(mu_acq)
std_acq = np.squeeze(std_acq)


def plot_acq_values(acq_values, fig_title='Example'):
    """
    Function to plot a list of acqusition values
    """
    acq_valeus_arg_max = np.argmax(acq_values)
    plt.clf()
    fig, axs = plt.subplots(2, 1)
    axs[0].set_title(fig_title)
    axs[0].plot(x_test_acq, fx_gt_acq, "r--", label="True (unknown)")
    axs[0].plot(x_train_acq, y_train_acq, 'k*', label="Observations")
    axs[0].plot(x_test_acq, mu_acq, "b", label="prediction")
    axs[0].fill_between(x=np.squeeze(x_test_acq), y1=mu_acq - 1.9600 * std_acq, y2=mu_acq + 1.9600 * std_acq, alpha=.2, fc="b")

    axs[1].fill_between(x=np.squeeze(x_test_acq), y1=0, y2=acq_values)
    axs[1].plot(x_test_acq[acq_valeus_arg_max], acq_values[acq_valeus_arg_max], 'kx', label="maximal acq_value")
    axs[1].legend()
    axs[1].set_ylabel(f" acq_values")
    
    plt.show()

    #plt.savefig(f"{fig_title}.png")
    
def plot_acq_values_wrapper(
                acq_func_class, 
                acq_func_init_kwargs, 
                acq_func_update_kwargs, 
                fig_title='Example'
    ):
    
    """
    Wrapper for plotting acquisition function values
    
    Parameters
    -----------

    acq_func_class  
        Class of acqusition function i.e the function definition
    
    acq_func_init_kwargs
        Initial arguments required to instantiate the acquisition function object                        
    
    acq_func_update_kwargs
        The parameters of the function that need to be updated on the fly 
        (each time when the model receives new data  and is retrained)
    
    fig_title 
        the title of the figure
    """
    acq_func = acq_func_class(**acq_func_init_kwargs)
    acq_func.update(**acq_func_update_kwargs)
    acq_values = acq_func(configs_test_acq)
    plot_acq_values(acq_values=acq_values, fig_title=fig_title)
    

### A dummy acquisition function

As before, we use a dummy acqusition function showing you how to correctly update the contents of the acquisition functions.

In [ ]:
class DummyAcqFunc(AbstractAcquisitionFunction):
    def __init__(self,
                 model: BaseModel):
        """
        This is only an example showing you how to correctly implement
        acquisition functions. It returns a randomly sampled value plus the 
        mean value predicted by the model.

        Thus, to compute acquisition function values correctly, you need to update 
        a new model that is fitted to the current data distribution before you can 
        actually compute the acquisitino function value

        Parameters
        -----------
        model 
            model to compute the data distribution
        
        """
        super(DummyAcqFunc, self).__init__(model)
        self._required_updates = ('model',)
    
    
    def _compute(self, X: np.ndarray) -> np.ndarray:
        if len(X.shape) == 1:
            X = X[:, np.newaxis]
        m, _ = self.model.predict(X)
        return np.random.randn(X.shape[0]) + m

In [ ]:
# To compute DummyAcqFunc values, only model is required. Thus `DummyAcqFunc._required_updates 
# only requires surro_model_acq as update keys. 
dummy_init_kwargs = {'model': surro_model_acq}
dummy_update_kwargs = {'model': surro_model_acq}

plot_acq_values_wrapper(
                DummyAcqFunc, 
                acq_func_init_kwargs=dummy_init_kwargs, 
                acq_func_update_kwargs=dummy_update_kwargs
            )

### Improvement-based Policies

Improvement-based acquisition functions select the points that are most likely to improve upon the incumbent at time step $t$ as: 

$$
\hat{\mathbf{\lambda}}^{(t-1)}\in argmin_{\mathbf{\lambda}' \in D^{(t-1)}}c(\mathbf{\lambda}')
$$ 

where $c(\lambda)$ is the cost of the hyperparameter configuration~$\lambda$. We write $c_{inc}$ as shorthand for the cost of the current incumbent: $c_{inc} = \min_{t' \leq t-1} c(\hat{\mathbf{\lambda}}^{(t')})$.

#### Probabilitiy of Improvement (PI)

The value of the acquisition function is proportional to the probability of improvement at each point. Probability of improvement (PI) is computed by

$$
\alpha_{PI}(\mathbf{x}; D_n) = P \big(c(\mathbf{\lambda}) \leq c_{inc} \big)
$$

Since the predictive distribution for $c(\lambda)$ is a Gaussian distribution,  $\mathcal{N} \big(\mu^{(t-1)}(\mathbf{\lambda}), {\sigma^2}^{(t-1)}(\mathbf{\lambda}) \big)$, this can be written as 
$u_{PI}^{(t)}(\mathbf{\lambda}) = \Phi(Z)$, with
\begin{equation}
    Z=\frac{c_{inc} - \mu^{(t-1)}(\mathbf{\lambda}) - \xi}{\sigma^{(t-1)}(\mathbf{\lambda})}
\end{equation}

where $\Phi(\cdot)$ is the CDF of the standard normal distribution and $\xi$ is an optional exploration hyperparameter, which needs to be specified with the initialization of the acquisition function.


In [ ]:
from scipy.stats import norm

class PI(AbstractAcquisitionFunction):
    #TODO: implment all the related function in PI, you need to consider which values are required for
    # computing PI values when new data comes


In [ ]:
# TODO update the values that are requried by computing a PI acquisition function

eta = np.min(y_train_acq)

# this is the dictionary of arguments that we use to initialize
pi_init_kwargs = {}

# this is the dictionary of arguments that need to be updated
pi_acq_update_kwargs = {}

plot_acq_values_wrapper(PI, 
                        acq_func_init_kwargs=pi_init_kwargs, 
                        acq_func_update_kwargs=pi_acq_update_kwargs)

#### Expected Improvement (EI)

The value is not just proportional to the probability, but also to the magnitude of possible improvement from the point. The one-step positive improvement over the current incumbent is defined as: 
$$
I^{(t)}(\mathbf{\lambda}) = \max (0, c_{inc} - c(\mathbf{\lambda}))
$$ 

Expected improvement (EI) is then defined as:

$$
\begin{align}
u_{EI}^{(t)}(\mathbf{\lambda})
&= \mathbb{E}[\mathbf{I}^{(t)}(\mathbf{\lambda})] \nonumber  \\
&= \int_{-\infty}^{\infty} p^{(t)}(c \mid \lambda) \cdot I^{(t)}(\lambda) \mathrm{d} c.
\end{align}
$$

Since the posterior distribution of $\hat{c}(\mathbf{\lambda})$ is a Gaussian, the Expected Imporvement (EI) can be computed in closed form:

$$
{u_{EI}^{(t)}(\mathbf{\lambda})} =
\begin{cases}
    \sigma^{(t)}(\mathbf{\lambda})[Z\Phi(Z) + \phi(Z)], & \text{if }\sigma^{(t)}(\mathbf{\lambda}) > 0 \\
    0 & \text{if }\sigma^{(t)}(\mathbf{\lambda}) = 0,
\end{cases}
$$

where $Z =\dfrac{c_{inc} - \mu^{(t)}(\mathbf{\lambda}) - \xi}{\sigma^{(t)}(\mathbf{\lambda})}$ and  $\xi$  is an optional exploration hyperparameter. As in the case of PI, $\xi$ needs to be specified with the initialization of the acquisition function.

In [ ]:
class EI(AbstractAcquisitionFunction):
        #TODO: implment all the related function in EI, you need to consider which values are required when 
        # new data comes

In [ ]:
# TODO update the values that are requried by computing a EI acquisition function

eta = np.min(y_train_acq)

ei_init_kwargs = {}
ei_acq_update_kwargs = {}

plot_acq_values_wrapper(EI, 
                        acq_func_init_kwargs=ei_init_kwargs, 
                        acq_func_update_kwargs=ei_acq_update_kwargs)

### Other Policies

Several other acquisition functions exist that do not compute the improvement upon the current incumbent, and so, these policies require different update rules

#### Lower  Confidence Bound (LCB)

We control the exploration through the variance and control parameter and exploit the minimum values. Lower confidence Bound (LCB) is defined by:

$$
u_{LCB}^{(t)} (\mathbf{\lambda}) - \alpha_t\sigma^{(t)}(\mathbf{\lambda})
$$

here $\alpha_t$ is given by:
$\alpha_t = \sqrt{(2\log (|D|t^2\pi^2/6\delta))}$, where $\delta$ is a constant value controlling the exploration-exploitation behaviour, in practice, all the constant values can be merged into one single value $\xi$ to reduce the computation overload. $|D|$ is the number of data point dimensions, $t$ is the number of evaluations, e.g. the number of data points, which needs to be updated each time when new values are observed.

In [ ]:
class LCB(AbstractAcquisitionFunction):
        #TODO: implment all the related function in LCB, you need to consider which values are required when 
        # new data comes

In [ ]:
# TODO update the values that are requried by computing a LCB acquisition function

num_data = np.shape(y_train_acq)[0]

lcb_init_kwargs = {}
lcb_acq_update_kwargs = {}

plot_acq_values_wrapper(LCB, 
                        acq_func_init_kwargs=lcb_init_kwargs, 
                        acq_func_update_kwargs=lcb_acq_update_kwargs)

In [ ]:
## Acquisition Function Optimizer


In the previous section, we maximized the acquisition function by doing a grid search: this might only work well for lower dimensional space. For higher dimensional problems, we need a more advanced streagy: an acquisition function optimizer that suggests a configuration with the best acquisition function value. 

We evaluate 20 different configurations and store the results under ```runhistory.json```. Starting from the already evaluated configurations, we need to ask our optimizer to give suggestions to be evaluated in the next iteration. Before digging deeper into Acqusition function optimizer, first you need to complete a BO iteration described at [Bayesian Optimization](#Bayesian-Optimization).


Just like before, we will plot the trajectory of the acquisition function, please plot all the points suggsted by the optimizer with cyan "x" marker and mark the point that has the largest acquisition function value (this configuration should also be the one that will be evaluated further) with black.

In [ ]:
from surrogate_model.gaussian_process import GaussianProcess

from matplotlib.colors import LogNorm

# we use EI to do illustration
model_opt = GaussianProcess()
acq_func = EI(model=model_opt)

# previous run histoy is stored here
rh = RunHistory()
rh.load_json('kmnist_opt/run_0/runhistory.json', cs_lenet)

# Transform rh to numpy array    
x_cfgs, y_cfgs, previous_cfgs = collect_evaluation_info(rh)
    

# plotting information
resolution = 150

x_grid_values = np.linspace(0, 1, resolution)
y_grid_values = np.linspace(0, 1, resolution)

x_ax, y_ax = np.meshgrid(x_grid_values, y_grid_values)

X_test = np.dstack([x_ax, y_ax]).reshape([-1, 2])


# TODO finish the first two step of one BO iteration: fit a surrogate model with the previous configurations
# and update all the information requried by computing an acquisition function



def plot_acq_optimizer(acq_optimier=None, opt_kwargs={}):
    # As before, we plot how the acq_optimizer gives the final suggestion
    acq_grid_values = acq_func._compute(X_test).reshape([resolution, resolution])

    fig, ax = plt.subplots()

    ax.plot(x_cfgs[:, 0], x_cfgs[:, 1], "x", color='r', markersize=3,
            lw=0, label="Previous Samples")
    
    if acq_optimier is not None:
        # TODO generate all the configurations suggested by the acquisition function maximizer and store them
        # as `cfgs_suggest`
        cfgs_suggest = []
        x_suggest = np.zeros([len(cfgs_suggest), len(cs_lenet.get_hyperparameters())])

        for idx_cfg, cfg_suggest in enumerate(cfgs_suggest):
            x_suggest[idx_cfg] = cfg_suggest.get_array()
        
        # TODO plot the configurations suggested by the optimizer


    cm = ax.pcolormesh(x_ax, y_ax, acq_grid_values.reshape([resolution, resolution]),
                       )

    cb = fig.colorbar(cm)

    plt.xlabel('batch_size')
    plt.ylabel('learning_rate')
    plt.legend(loc='upper right')
    plt.title(type(acq_func).__name__)
    plt.show()
    
plot_acq_optimizer()

First let us define an abstract class that provides a uniform API for all the optimzers

In [ ]:
import abc


class AcquisitionFunctionMaximizer(object, metaclass=abc.ABCMeta):
    """
    Abstract class for acquisition maximization.
    In order to use this class it has to be subclassed and the method
    ``_maximize`` must be implemented.
    """
    def __init__(
            self,
            acquisition_function: AbstractAcquisitionFunction,
            config_space: ConfigurationSpace,
            rng: typing.Optional[np.random.RandomState] = None,
    ):
        self.acquisition_function = acquisition_function
        self.config_space = config_space

        if rng is None:
            self.rng = np.random.RandomState(seed=1)
        else:
            self.rng = rng

    def maximize(
        self,
        previous_configs: typing.List[Configuration],
        **kwargs,
    ) -> typing.Iterator[Configuration]:
        """
        Maximize acquisition function using ``_maximize``.
        Parameter
        -------
        previous_configs: List[Configuration]
            Configurations that are evaluated in the previous runs

        Returns
        -------
        iterable
            An iterable consisting of :class:`configspace.Configuration`.
        """
        return iter([t[1] for t in self._maximize(previous_configs, **kwargs)])

    @abc.abstractmethod
    def _maximize(
            self,
            previous_configs: typing.List[Configuration],
            num_points: int,
    ) -> typing.List[typing.Tuple[float, Configuration]]:
        """
        Implements acquisition function maximization.
        In contrast to ``maximize``, this method returns an iterable of tuples,
        consisting of the acquisition function value and the configuration. This
        allows to plug together different acquisition function maximizers.
        Returns
        -------
        iterable
            An iterable consistng of
            tuple(acqusition_value, :class:`configspace.Configuration`).
        """
        raise NotImplementedError()

    def _sort_configs_by_acq_value(
            self,
            configs: typing.List[Configuration]
    ) -> typing.List[typing.Tuple[float, Configuration]]:
        """
        Sort the given configurations by acquisition value
        Parameters
        ----------
        configs : list(Configuration)
        Returns
        -------
        list: (acquisition value, Candidate solutions),
                ordered by their acquisition function value
        """
        acq_values = self.acquisition_function(configs)

        # From here
        # http://stackoverflow.com/questions/20197990/how-to-make-argsort-result-to-be-random-between-equal-values
        random = self.rng.rand(len(acq_values))
        # Last column is primary sort key!
        indices = np.lexsort((random.flatten(), acq_values.flatten()))

        # Cannot use zip here because the indices array cannot index the
        # rand_configs list, because the second is a pure python list
        return [(acq_values[ind], configs[ind]) for ind in indices[::-1]]

### Random Optimizer

Due to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality), the number of points required by grid search becomes almost prohibitive under high dimensional search spaces. Furthermore, when some hyperparameters are much more important than others, random search often performs much better than grid search. Thus, random search can better solve these kinds of problems. The number of the samples is specified by the users while the configuration values of each dimension are (potentially) different. Please sample 20 points from the configuration space and plot the trajectory.

In [ ]:
class RandomOptimizer(AcquisitionFunctionMaximizer):
    def _maximize(
            self,
            previous_configs: typing.List[Configuration],
            num_points: int,
    ) -> typing.List[typing.Tuple[float, Configuration]]:
        # TODO randomly sample num_points configuratinos and then return the configurations that are sorted by their
        # acquisition function values (You could find the corresponding function by the base Class)

In [ ]:
rand_opt = RandomOptimizer(acquisition_function=acq_func, config_space=cs_lenet)
opt_kwargs = {'num_points': 20}
plot_acq_optimizer(rand_opt, opt_kwargs)

### Sobol Sequence

Despite its strong performance, random search cannot guarantee that the search space is uniformly covered by the sampled points. A [Sobol Sequence](https://math.stackexchange.com/questions/888490/understanding-sobol-sequences) covers the whole search space regardless the number of the search space dimensions. Please select 1000 points of a Sobol sequence using [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html) (we want to sample an arbitrary number of points) and  plot the trajectory. You need to use the function `transform_continuous_designs` under [util](#Acquisition-Function-Optimizer) to transform teh numpy array to Configuration objects.

In [ ]:
from scipy.stats.qmc import Sobol

class SobolOptimizer(AcquisitionFunctionMaximizer):
    # TODO implement all the functions that are required to for sobol optimizer 

In [ ]:
sobol_opt = SobolOptimizer(acquisition_function=acq_func, config_space=cs_lenet)
opt_kwargs = {'num_points': 20}
plot_acq_optimizer(sobol_opt, opt_kwargs)

### Local Optimizer


Global search provides a coarse estimation of the global search space. We can then fine tune the hyperparameter configurations with local search. Starting from a given set of candidates, local search iteratively moves to the neighbors of the candidates to find the optimal solutions. 

You can call the method `ConfigSpace.util.get_one_exchange_neighbourhood()` (you can specify the corrected values by yourself or use the function `get_one_exchange_neighbourhood()` under [util](#Acquisition-Function-Optimizer) defined below to get a one-exchange neighborhood. You should then iteratively select the neighbors that improve upon the incumbent candidates. The search stops if none of the neighbors have better acquisition values compared to the current candidates (Sometimes, it is still beneficial to make some plateau moves).

Local search requires a start points. Please start from 10 previous configurations with the largest acquisition function values and do a local search starting from each point. Besides that, the user could also specify a set of points `additional_start_points` as starting points (in which case, both starting points sets contain `num_points` starting points). This time you only need to return the point suggested by each local search trajectory.

In [ ]:
import itertools

class LocalSearchOptimizer(AcquisitionFunctionMaximizer):
    def _maximize(
            self,
            previous_configs: typing.List[Configuration],
            num_points: int,
            additional_start_points: typing.Optional[typing.List[typing.Tuple[float, Configuration]]] = None,
    ) -> typing.List[typing.Tuple[float, Configuration]]:
        init_points = self._get_initial_points(num_points=10,
                                               previous_configs=previous_configs,
                                               additional_start_points=additional_start_points)
        configs_acq = self._do_search(init_points)

        # shuffle for random tie-break
        self.rng.shuffle(configs_acq)

        # sort according to acq value
        configs_acq.sort(reverse=True, key=lambda x: x[0])
        return configs_acq

    def _get_initial_points(
            self,
            num_points: int,
            previous_configs: typing.List[Configuration],
            additional_start_points: typing.Optional[typing.List[typing.Tuple[float, Configuration]]],
    ) -> typing.List[Configuration]:
        raise NotImplementedError

        # TODO implement all the functions that are required to for sobol optimizer, you need to consider how to get the
        # initial starting points and how to perform local search

    def _do_search(
            self,
            start_points: typing.List[Configuration],
    ) -> typing.List[typing.Tuple[float, Configuration]]:
        raise NotImplementedError

In [ ]:
local_opt = LocalSearchOptimizer(acquisition_function=acq_func, config_space=cs_lenet)
opt_kwargs = {'num_points': 10}
plot_acq_optimizer(local_opt, opt_kwargs)

### Global and Local Search

Local search could also start from a point suggested by a global search. Please implement an algorithm that combines the global and random search, e.g. first you should use global search to select the most promising candidates, then these candidates together with the current best observations are passed as starting points to your local search algorithms. As before, `num_points` is set as 10.

In [ ]:
class GlobalAndLocalSerchOptimizer(AcquisitionFunctionMaximizer):
    def __init__(
            self,
            acquisition_function: AbstractAcquisitionFunction,
            config_space: ConfigurationSpace,
            rng: typing.Union[bool, np.random.RandomState] = None,
            global_optimizer: typing.Optional[AcquisitionFunctionMaximizer] = None,
            local_optimizer: typing.Optional[AcquisitionFunctionMaximizer] = None,
    ):
        # TODO consider how to combine global and local optimizer peacefully

        raise NotImplementedError



In [ ]:
local_opt = GlobalAndLocalSerchOptimizer(acquisition_function=acq_func, config_space=cs_lenet)
opt_kwargs = {'num_points': 10, 'num_points_global': 20}
plot_acq_optimizer(local_opt, opt_kwargs)

# Evaluation

 Finally, choose one of your acquisition functions and surrogate models to continue the hyperparameter optimization of the neural network for 5 more iterations. Plot the incumbent loss with respect to the number of evaluations (you need to plot the losses of all the previously evaluated functions). You might need some functions defined under [util](#Acquisition-Function-Optimizer)

In [ ]:
# Suggestion: a new configuration with its evaluated cost can be added to runhistory with the following
# examplary block:
import time
import copy
from smac.tae import StatusType


def eval_cfg_with_time_info(cfg_lenet):
    time_start = time.time()
    val_score = evaluator.eval_config(cfg_lenet)
    time_end = time.time()
    return val_score, time_end - time_start

"""
# The following code is only an example showing how to add new run results to the runhistory

# we don't want to add this cfg to the raw runhistory
rh_copy = copy.deepcopy(rh)

cfg_random = cs_lenet.sample_configuration()
val_score, uses_time = eval_cfg_with_time_info(cfg_random)


rh_copy.add(cfg_random, val_score, uses_time, StatusType.SUCCESS)
"""

In [ ]:
def do_bayesian_optimization(target_func: typing.Callable,
                             rh: RunHistory,
                             config_space: ConfigurationSpace,
                             model: typing.Optional[BaseModel]=None,
                             acq_func: typing.Optional[AbstractAcquisitionFunction]=None,
                             acq_optimizer:typing.Optional[AcquisitionFunctionMaximizer]=None) -> RunHistory:

    if model is None:
        model = GaussianProcess()
    if acq_func is None:
        acq_func = EI(model)
    if acq_optimizer is None:
        acq_optimizer = GlobalAndLocalSerchOptimizer(acquisition_function=acq_func, config_space=config_space)
    
    if isinstance(acq_optimizer, (RandomOptimizer, SobolOptimizer)):
        optimizer_kwargs = {"num_points": 1000} 
    elif isinstance(acq_optimizer, LocalSearchOptimizer):
        optimizer_kwargs = {"num_points": 10} 
    elif isinstance(acq_optimizer, GlobalAndLocalSerchOptimizer):
        optimizer_kwargs = {"num_points": 10, "num_points_global": 1000} 
    else: raise ValueError(f'Unsupported type of acq_optimizer: {acq_optimizer}')

    
    for _ in range(5):
        # TODO implement a full BO loop here
        raise NotImplementedError
    return rh

In [ ]:
rh = do_bayesian_optimization(target_func=eval_cfg_with_time_info, 
                              rh=rh,
                              config_space=cs_lenet)

In [ ]:
_, y, _ = collect_evaluation_info(rh)


# TODO: Plot the incumbent loss values
